In [1]:
import csv
from datetime import datetime
import glob
import os
from io import StringIO
import requests
import pytz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint
from turtle import position

### 訓練集資料，資料年份:2022/01~2023/12

In [2]:
fwd_path = os.path.abspath(os.path.join(os.getcwd(), "..")) #取得當前工作目錄的上一集目錄，並存入路徑
stock_num = ['UNIUSDT_期貨','UNIUSDT_現貨','DYDXUSDT_期貨','DYDXUSDT_現貨']
df = pd.DataFrame()
for i in stock_num:
    file_path = "C:/python/Crypto/Train_Set/"+ i +".csv"
    file = pd.read_csv(file_path, index_col = 'open_time')
    df = df.dropna()
    file = pd.DataFrame(file['close'])
    df = df.dropna()
    df = pd.concat([df, file], axis = 1)
    df = df.dropna()
df.columns = ['UNIUSDT期貨價格','UNIUSDT現貨價格','DYDXUSDT期貨價格','DYDXUSDT現貨價格']
df = df.dropna()

df['UNIUSDT_Spread'] = df['UNIUSDT期貨價格'] / df['UNIUSDT現貨價格']
df['DYDXUSDT_Spread'] = df['DYDXUSDT期貨價格'] / df['DYDXUSDT現貨價格']
df['Target_Spread'] = df['UNIUSDT_Spread'] / df['DYDXUSDT_Spread']
df['SMA'] = df['Target_Spread'].rolling(60).mean()
df['STD'] = df['Target_Spread'].rolling(60).std()
df['Z'] = (df['Target_Spread'] - df['SMA']) / df['STD']
df = df.dropna()
print(df)

                     UNIUSDT期貨價格  UNIUSDT現貨價格  DYDXUSDT期貨價格  DYDXUSDT現貨價格  \
open_time                                                                   
2022-01-01 00:59:00       18.019       18.040         8.403         8.412   
2022-01-01 01:00:00       18.038       18.030         8.429         8.433   
2022-01-01 01:01:00       18.032       18.030         8.446         8.452   
2022-01-01 01:02:00       18.032       18.030         8.455         8.462   
2022-01-01 01:03:00       18.028       18.030         8.465         8.470   
...                          ...          ...           ...           ...   
2023-12-31 20:35:00        7.683        7.671         3.094         3.091   
2023-12-31 20:36:00        7.684        7.673         3.094         3.092   
2023-12-31 20:37:00        7.666        7.657         3.092         3.088   
2023-12-31 20:38:00        7.663        7.655         3.093         3.089   
2023-12-31 20:39:00        7.673        7.665         3.092         3.089   

## ADF檢定

In [3]:
def adf_test(series, title=''):
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic', 'p-value', 'Number of lags used', 'Number of observations used']
    out = pd.Series(result[0:4], index=labels)
    for k, v in result[4].items():
        out[f'critical value ({k})'] = v
    print(out)
    if result[1] <= 0.05:  # 有顯著性，推翻虛無假設
        print("Data has no unit root and is stationary")
    else:
        print("Data has a unit root and is non-stationary")

In [4]:
columns_to_test = ['Z']  # 指定要測試的 column 名稱列表

for col in columns_to_test:
    print(f"Series is from column: {col}")
    adf_test(df[col], title=col)
    print()

Series is from column: Z
Augmented Dickey-Fuller Test: Z
ADF test statistic               -102.309539
p-value                             0.000000
Number of lags used               110.000000
Number of observations used    700830.000000
critical value (1%)                -3.430359
critical value (5%)                -2.861544
critical value (10%)               -2.566772
dtype: float64
Data has no unit root and is stationary

